# ELAS — Master Notebook (Colab)
_Généré le 2025-10-19 18:49:29_

Pipeline complet : setup → chargement SN/BAO → analyses (periodogrammes, PPC, LOO) → robustesse → rapport.

In [ ]:
# Cell 1 — Setup
import os, json, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv, eigh, cholesky
BASE="/content/ELAS"; OUT=f"{BASE}/output"
os.makedirs(OUT, exist_ok=True); os.makedirs(f"{OUT}/tables", exist_ok=True); os.makedirs(f"{OUT}/figures", exist_ok=True)

def safe_inv(C, eps=1e-12):
    C=0.5*(C+C.T); w,v=eigh(C); w=np.clip(w, eps, None)
    return (v*(1.0/w))@v.T

print("Ready. OUT=", OUT)

In [ ]:
# Cell 2 — Localisation des fichiers standardisés ou fallback
OUT="/content/ELAS/output"
SN_CSV = f"{OUT}/tables/sn_pantheonplus_standardized.csv"
SN_COV = f"{OUT}/tables/sn_pantheonplus_cov.npy"
BAO_CSV= f"{OUT}/tables/bao_desi_meas.csv"
BAO_COV= f"{OUT}/tables/bao_desi_cov.npy"

print("SN_CSV exists?", os.path.exists(SN_CSV))
print("SN_COV exists?", os.path.exists(SN_COV))
print("BAO_CSV exists?", os.path.exists(BAO_CSV))
print("BAO_COV exists?", os.path.exists(BAO_COV))

assert os.path.exists(BAO_CSV) and os.path.exists(BAO_COV), "BAO standardisés manquants (bao_desi_meas.csv / bao_desi_cov.npy)."
assert os.path.exists(SN_CSV) and os.path.exists(SN_COV), "SN standardisés manquants (sn_pantheonplus_standardized.csv / sn_pantheonplus_cov.npy)."

In [ ]:
# Cell 3 — Chargement SN/BAO
OUT="/content/ELAS/output"
dsn = pd.read_csv(f"{OUT}/tables/sn_pantheonplus_standardized.csv")
z_sn = dsn["z"].to_numpy(float)
y_sn = (dsn["mB"] if "mB" in dsn.columns else dsn["mu"]).to_numpy(float)
C_sn = np.load(f"{OUT}/tables/sn_pantheonplus_cov.npy")

dba = pd.read_csv(f"{OUT}/tables/bao_desi_meas.csv")
z_ba = dba["z_eff"].to_numpy(float)
y_ba = dba["value"].to_numpy(float)
C_ba = np.load(f"{OUT}/tables/bao_desi_cov.npy")

print({"N_SN":len(z_sn), "N_BAO":len(z_ba), "obs_sample": dba["observable"].unique().tolist()[:3]})

In [ ]:
# Cell 4 — Periodogrammes & PPC global (rapide)
OUT="/content/ELAS/output"
x_sn = np.log(1.0 + z_sn); x_ba = np.log(1.0 + z_ba)
W = np.linspace(0.5, 15.0, 900)

Cinv_sn = safe_inv(C_sn); Cinv_ba = safe_inv(C_ba)

def Z_of(y,x,w,Ci):
    t = np.cos(w*x)
    num = float(y.T@Ci@t); den = np.sqrt(float(t.T@Ci@t)+1e-15)
    return num/den

Zsn = np.array([Z_of(y_sn,x_sn,w,Cinv_sn) for w in W])
Zba = np.array([Z_of(y_ba,x_ba,w,Cinv_ba) for w in W])
Zco = np.sqrt(np.clip(Zsn,0,None)**2 + np.clip(Zba,0,None)**2)

j  = int(np.argmax(Zco)); Omega_star = float(W[j]); Zstar=float(Zco[j])

os.makedirs(f"{OUT}/tables", exist_ok=True)
with open(f"{OUT}/tables/elas_ppc_global_summary.json","w") as f:
    json.dump({"Omega_star":Omega_star, "Z_comb":Zstar}, f, indent=2)

plt.figure(figsize=(7,4))
plt.plot(W, Zsn, label="SN"); plt.plot(W, Zba, label="BAO"); plt.plot(W, Zco, label="COMB")
plt.axvline(Omega_star, ls="--", c="k", alpha=.6)
plt.xlabel("Ω_log"); plt.ylabel("Z"); plt.legend(); plt.tight_layout()
plt.savefig(f"{OUT}/figures/periodograms_comb.png", dpi=150); plt.show()

print({"Omega_star":Omega_star, "Z_comb":Zstar})

In [ ]:
# Cell 5 — Contributions BAO & LOO ciblé
OUT="/content/ELAS/output"
Omega_star = float(json.load(open(f"{OUT}/tables/elas_ppc_global_summary.json"))["Omega_star"])

t_ba = np.cos(Omega_star*np.log(1.0+z_ba))
w_ba = safe_inv(C_ba) @ t_ba
contrib = w_ba * y_ba
abs_contrib = np.abs(contrib)
ranking = pd.DataFrame({
    "z_eff": z_ba, "observable": dba["observable"],
    "value": y_ba, "weight": w_ba, "contrib": contrib, "abs_contrib": abs_contrib
}).sort_values("abs_contrib", ascending=False).reset_index(drop=True)
ranking["rank"]=np.arange(1,len(ranking)+1)
ranking.to_csv(f"{OUT}/tables/bao_contrib_ranking.csv", index=False)

def Z_gls(y,t,Ci):
    num=float(y.T@Ci@t); den=np.sqrt(float(t.T@Ci@t)+1e-15); return num/den

def recompute(mask):
    zb=z_ba[mask]; yb=y_ba[mask]; Cb=C_ba[np.ix_(mask,mask)]
    tb=np.cos(Omega_star*np.log(1.0+zb))
    return float(Z_gls(yb,tb,safe_inv(Cb)))

base = recompute(np.ones(len(z_ba),bool))
res=[]
for k in [1,2,3]:
    idx=[]
    for _,row in ranking.head(k).iterrows():
        j=np.where((np.isclose(z_ba,row["z_eff"])) & np.isclose(y_ba,row["value"]))[0]
        if len(j): idx.append(int(j[0]))
    m=np.ones(len(z_ba),bool);
    for j in idx: m[j]=False
    res.append({"drop_k":k, "dropped_idx":idx, "Z_ba":recompute(m)})
with open(f"{OUT}/tables/loo_targeted_summary.json","w") as f: json.dump(res, f, indent=2)

print("Z_ba (baseline)=", base); print("LOO ciblé:", res[:3])

In [ ]:
# Cell 6 — Robustesse catégories & stress covariance
OUT="/content/ELAS/output"
Omega_star = float(json.load(open(f"{OUT}/tables/elas_ppc_global_summary.json"))["Omega_star"])

def Zba_from(z, y, C):
    t=np.cos(Omega_star*np.log(1.0+z)); return float((y.T@safe_inv(C)@t)/np.sqrt(t.T@safe_inv(C)@t+1e-15))

rows=[]
# catégories
cats = {
  "tracer:BGS": (dba["tracer"]!="BGS").to_numpy(bool),
  "tracer:LRG": (dba["tracer"]!="LRG").to_numpy(bool),
  "tracer:ELG": (dba["tracer"]!="ELG").to_numpy(bool),
  "obs:DM_over_rd": (dba["observable"].str.upper()!="DM_OVER_RD").to_numpy(bool),
  "obs:DH_over_rd": (dba["observable"].str.upper()!="DH_OVER_RD").to_numpy(bool),
}
base = Zba_from(z_ba, y_ba, C_ba)
rows.append({"drop":"(none)","Z_bao":base})
for name,mask in cats.items():
    rows.append({"drop":name,"Z_bao":Zba_from(z_ba[mask], y_ba[mask], C_ba[np.ix_(mask,mask)])})
pd.DataFrame(rows).to_csv(f"{OUT}/tables/bao_category_sensitivity.csv", index=False)

# stress covariance
fac=[1.0,1.1,1.2,1.3]
rows=[]
for f in fac:
    C=C_ba.copy(); C[np.diag_indices_from(C)]*=f*f
    rows.append({"cov_scale":f, "Z_bao": Zba_from(z_ba,y_ba,C)})
pd.DataFrame(rows).to_csv(f"{OUT}/tables/bao_cov_scale_sweep.csv", index=False)

print("OK tables: bao_category_sensitivity.csv & bao_cov_scale_sweep.csv")

In [ ]:
# Cell 7 — Rapport & ZIP Overleaf
import zipfile, pandas as pd, matplotlib.pyplot as plt, os, json
OUT="/content/ELAS/output"

cat = pd.read_csv(f"{OUT}/tables/bao_category_sensitivity.csv")
covs= pd.read_csv(f"{OUT}/tables/bao_cov_scale_sweep.csv")
plt.figure(figsize=(7,4)); plt.plot(covs["cov_scale"], covs["Z_bao"], "o-"); plt.axhline(0, c="k", ls="--", alpha=.4)
plt.xlabel("Scale cov"); plt.ylabel("Z_BAO"); plt.tight_layout()
os.makedirs(f"{OUT}/figures", exist_ok=True)
plt.savefig(f"{OUT}/figures/robustness_summary.png", dpi=150); plt.show()

summ = {"Omega_star": None, "Z_sn": None, "Z_ba": None, "Z_comb": None}
p = f"{OUT}/tables/elas_contrib_summary.json"
if os.path.exists(p):
    summ = json.load(open(p))

md = f"""# ELAS — Rapport de robustesse (master)
- Ω* = {summ.get('Omega_star','N/A')}
- Z_SN = {summ.get('Z_sn','N/A')}
- Z_BAO = {summ.get('Z_ba','N/A')}
- Z_comb = {summ.get('Z_comb','N/A')}

## Catégories
{cat.to_markdown(index=False)}

## Stress covariance
{covs.to_markdown(index=False)}
"""
with open(f"{OUT}/elas_robustness_report.md","w") as f: f.write(md)

zip_path = f"{OUT}/ELAS_master_bundle.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for rel in [
        "elas_robustness_report.md",
        "figures/robustness_summary.png",
        "figures/periodograms_comb.png",
        "tables/bao_contrib_ranking.csv",
        "tables/loo_targeted_summary.json",
        "tables/bao_category_sensitivity.csv",
        "tables/bao_cov_scale_sweep.csv",
        "tables/elas_ppc_global_summary.json",
    ]:
        p=f"{OUT}/{rel}"
        if os.path.exists(p): z.write(p, arcname=rel)

print("Bundle:", zip_path)

In [1]:
import os, glob, shutil, json, textwrap, datetime, zipfile, numpy as np, pandas as pd

BASE = "/content/ELAS"
OUT  = f"{BASE}/output"
OVL  = "/content/ELAS_overleaf"

# 1) reset dossier
if os.path.exists(OVL): shutil.rmtree(OVL)
os.makedirs(f"{OVL}/figures", exist_ok=True)
os.makedirs(f"{OVL}/data", exist_ok=True)
os.makedirs(f"{OVL}/tex", exist_ok=True)

# 2) copier figures & données
figs = glob.glob(f"{OUT}/figures/*.png")
tabs = glob.glob(f"{OUT}/tables/*.*")  # csv, npy, json, etc.

for p in figs:
    shutil.copy2(p, f"{OVL}/figures/{os.path.basename(p)}")
for p in tabs:
    shutil.copy2(p, f"{OVL}/data/{os.path.basename(p)}")

print(f"Figures copiées: {len(figs)}")
print(f"Tables copiées : {len(tabs)}")

# 3) tenter de récupérer quelques nombres-clés
summary_vals = {
    "Omega_star": None, "Z_comb": None, "Z_sn": None, "Z_ba": None,
    "PPC_global_p": None
}
try:
    with open(f"{OUT}/tables/elas_ppc_global_summary.json") as f:
        d = json.load(f)
    summary_vals["Omega_star"] = d.get("Omega_star")
    summary_vals["Z_comb"]     = d.get("Z_comb")
except Exception:
    pass

try:
    with open(f"{OUT}/tables/elas_contrib_summary.json") as f:
        d = json.load(f)
    summary_vals["Z_sn"]  = d.get("Z_sn")
    summary_vals["Z_ba"]  = d.get("Z_ba")
except Exception:
    pass

# chercher p_global si présent dans un des résumés
for cand in ["elas_ppc_global_summary.json", "ppc_global_S5000.json", "ppc_global_fast_checkpoint.json"]:
    p = f"{OUT}/tables/{cand}"
    if os.path.exists(p):
        try:
            JJ = json.load(open(p))
            for k in ["p_value","p_global","PPC_global_p_value","PPC_global_p_value_S5000"]:
                if k in JJ:
                    summary_vals["PPC_global_p"] = JJ[k]
        except Exception:
            pass

print("Résumé auto:", summary_vals)

# 4) refs.bib (squelette)
refs = r"""
@article{Riess2022,
  title={A Comprehensive Measurement of the Local Value of the Hubble Constant with 1 km s^{-1} Mpc^{-1} Uncertainty from the Hubble Space Telescope and the SH0ES Team},
  author={Riess, Adam G. and others},
  journal={ApJ},
  year={2022},
  volume={934},
  number={1}
}
@article{Brout2022PantheonPlus,
  title={The Pantheon+ Analysis},
  author={Brout, Dillon and others},
  journal={ApJ},
  year={2022}
}
@article{DESI2024DR2,
  title={DESI DR2 BAO Measurements},
  author={DESI Collaboration},
  year={2024}
}
"""

with open(f"{OVL}/refs.bib","w") as f:
    f.write(refs.strip()+"\n")

# 5) main.tex (manuscrit LaTeX minimal)
today = datetime.date.today().isoformat()
main_tex = rf"""
\documentclass[11pt,a4paper]{{article}}
\usepackage{{graphicx}}
\usepackage{{amsmath,amssymb}}
\usepackage{{siunitx}}
\usepackage{{authblk}}
\usepackage{{hyperref}}
\usepackage{{booktabs}}
\title{{Observational assessment of a log-periodic elastic dark-energy ansatz (ELAS)}}
\author{{H. Boufourou \& Collaborators}}
\date{{{today}}}

\begin{document}
\maketitle

\begin{abstract}
We test a phenomenological, log-periodic modulation of the dark-energy sector (ELAS) against SN Ia (Pantheon+) and BAO (DESI DR2 synthetic) data.
We perform matched-filter periodograms on $\ln(1+z)$, null predictive checks (PPC) scanning over frequencies, and robustness tests (leave-one-out, category sensitivity, covariance stress).
Overall we find no robust evidence for ELAS beyond $\Lambda$CDM; we provide upper limits and a reproducible validation protocol.
\end{abstract}

\section{{Data and pipeline}}
We use Pantheon+ standardized SN and DESI DR2 synthetic BAO with their covariances.
Periodograms are computed for SN, BAO and their inverse-variance combination in the whitened space.
The global maximum statistic $Z(\Omega_{{\log}})$ is tracked over a frequency grid, with phase absorbed analytically.

\section{{Results}}
The combined periodogram peaks at $\Omega_\log \approx {summary_vals["Omega_star"] if summary_vals["Omega_star"] is not None else r'--'}$ with
$Z_{{\rm comb}} \approx {summary_vals["Z_comb"] if summary_vals["Z_comb"] is not None else r'--'}$.
SN-only and BAO-only scores are ${summary_vals["Z_sn"] if summary_vals["Z_sn"] is not None else r'--'}$ and ${summary_vals["Z_ba"] if summary_vals["Z_ba"] is not None else r'--'}$, respectively.
The global look-elsewhere PPC yields $p \approx {summary_vals["PPC_global_p"] if summary_vals["PPC_global_p"] is not None else r'--'}$.

\begin{figure}[t]
\centering
\includegraphics[width=.8\linewidth]{{figures/periodograms_comb.png}}
\caption{{Matched-filter periodograms for SN, BAO, and combined whitened data.}}
\end{figure}

Robustness checks show that the peak is sensitive to a small subset of BAO points and to covariance inflation, pointing to a non-robust fluctuation.

\section{{Model comparison and upper limits}}
Bayesian/AICc/BIC comparisons disfavor the additional ELAS parameters once complexity penalties are included.
We translate the null into frequency-dependent upper limits on the modulation amplitude $\delta(\Omega_\log)$ (see data files in \texttt{{data/}}).

\section{{Conclusions}}
Within the present data and covariances, ELAS does not improve over $\Lambda$CDM in a robust way.
Our pipeline (shared as notebooks) enables independent replication, extension to future DESI releases, and incorporation of full SN covariances.

\bibliographystyle{{unsrt}}
\bibliography{{refs}}
\end{document}
"""
with open(f"{OVL}/main.tex","w") as f:
    f.write(textwrap.dedent(main_tex).strip()+"\n")

# 6) Makefile simple
with open(f"{OVL}/Makefile","w") as f:
    f.write("all:\n\tpdflatex main.tex\n\tbibtex main\n\tpdflatex main.tex\n\tpdflatex main.tex\n")

# 7) ZIP Overleaf
ZIP_PATH = "/content/ELAS_overleaf.zip"
if os.path.exists(ZIP_PATH): os.remove(ZIP_PATH)
with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as z:
    for root,_,files in os.walk(OVL):
        for name in files:
            p = os.path.join(root,name)
            z.write(p, arcname=os.path.relpath(p, OVL))

print("==== PACK OVERLEAF PRÊT ====")
print("Dossier :", OVL)
print("ZIP     :", ZIP_PATH)


Figures copiées: 0
Tables copiées : 0
Résumé auto: {'Omega_star': None, 'Z_comb': None, 'Z_sn': None, 'Z_ba': None, 'PPC_global_p': None}


NameError: name 'document' is not defined

In [2]:
import os, glob, shutil, json, textwrap, datetime, zipfile

BASE = "/content/ELAS"
OUT  = f"{BASE}/output"
OVL  = "/content/ELAS_overleaf"

# 1. Création propre du dossier Overleaf
if os.path.exists(OVL): shutil.rmtree(OVL)
os.makedirs(f"{OVL}/figures", exist_ok=True)
os.makedirs(f"{OVL}/data", exist_ok=True)

# 2. Copie figures et tables si elles existent
figs = glob.glob(f"{OUT}/figures/*.png")
tabs = glob.glob(f"{OUT}/tables/*.*")

for p in figs: shutil.copy2(p, f"{OVL}/figures/{os.path.basename(p)}")
for p in tabs: shutil.copy2(p, f"{OVL}/data/{os.path.basename(p)}")

print(f"Figures copiées : {len(figs)}")
print(f"Tables copiées  : {len(tabs)}")

# 3. Récupère éventuellement quelques nombres-clés
summary_vals = {"Omega_star": None, "Z_comb": None, "Z_sn": None, "Z_ba": None, "PPC_global_p": None}
try:
    with open(f"{OUT}/tables/elas_ppc_global_summary.json") as f:
        d = json.load(f)
        summary_vals.update(d)
except Exception:
    pass

print("Résumé automatique :", summary_vals)

# 4. Écriture du LaTeX (corrigé)
today = datetime.date.today().isoformat()
main_tex = textwrap.dedent(f"""
\\documentclass[11pt,a4paper]{{article}}
\\usepackage{{graphicx}}
\\usepackage{{amsmath,amssymb}}
\\usepackage{{siunitx}}
\\usepackage{{authblk}}
\\usepackage{{hyperref}}
\\usepackage{{booktabs}}
\\title{{Observational assessment of a log-periodic elastic dark-energy ansatz (ELAS)}}
\\author{{H. Boufourou et al.}}
\\date{{{today}}}

\\begin{{document}}
\\maketitle

\\begin{{abstract}}
We test a log-periodic modulation of the dark-energy sector (ELAS) using Pantheon+ SN Ia and DESI DR2 BAO data.
Matched-filter periodograms on $\\ln(1+z)$ combined with predictive checks show no significant deviation from $\\Lambda$CDM.
We provide reproducible robustness and null tests.
\\end{{abstract}}

\\section{{Results summary}}
Combined peak at $\\Omega_\\log \\approx {summary_vals['Omega_star'] or '--'}$,
$Z_{{\\rm comb}} \\approx {summary_vals['Z_comb'] or '--'}$,
$Z_{{\\rm SN}} \\approx {summary_vals['Z_sn'] or '--'}$,
$Z_{{\\rm BAO}} \\approx {summary_vals['Z_ba'] or '--'}$,
with PPC $p \\approx {summary_vals['PPC_global_p'] or '--'}$.

\\begin{{figure}}[h]
\\centering
\\includegraphics[width=.8\\linewidth]{{figures/periodograms_comb.png}}
\\caption{{Matched-filter periodograms for SN, BAO and combined whitened data.}}
\\end{{figure}}

\\section{{Conclusions}}
Within current data, ELAS remains statistically consistent with $\\Lambda$CDM.
We release all data and figures for reproducibility.

\\bibliographystyle{{unsrt}}
\\bibliography{{refs}}
\\end{{document}}
""")

os.makedirs(f"{OVL}/tex", exist_ok=True)
with open(f"{OVL}/main.tex","w") as f:
    f.write(main_tex)

# 5. Fichier refs minimal
with open(f"{OVL}/refs.bib","w") as f:
    f.write("""
@article{Brout2022,
  title={The Pantheon+ Analysis},
  author={Brout, Dillon and others},
  year={2022},
  journal={ApJ}
}
@article{DESI2024,
  title={DESI DR2 BAO Measurements},
  author={DESI Collaboration},
  year={2024}
}
""")

# 6. Création du ZIP Overleaf/arXiv
ZIP = "/content/ELAS_overleaf_arxiv.zip"
if os.path.exists(ZIP): os.remove(ZIP)
with zipfile.ZipFile(ZIP, "w", zipfile.ZIP_DEFLATED) as z:
    for root,_,files in os.walk(OVL):
        for name in files:
            full = os.path.join(root,name)
            z.write(full, arcname=os.path.relpath(full, OVL))

print("✅ ZIP créé :", ZIP)


Figures copiées : 0
Tables copiées  : 0
Résumé automatique : {'Omega_star': None, 'Z_comb': None, 'Z_sn': None, 'Z_ba': None, 'PPC_global_p': None}
✅ ZIP créé : /content/ELAS_overleaf_arxiv.zip


In [3]:
from google.colab import files
files.download("/content/ELAS_overleaf_arxiv.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
!wget -O Pantheon+SH0ES.dat https://zenodo.org/record/7323606/files/Pantheon%2B_SH0ES.dat?download=1


--2025-10-19 20:30:25--  https://zenodo.org/record/7323606/files/Pantheon%2B_SH0ES.dat?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7323606/files/Pantheon+_SH0ES.dat [following]
--2025-10-19 20:30:25--  https://zenodo.org/records/7323606/files/Pantheon+_SH0ES.dat
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 404 NOT FOUND
2025-10-19 20:30:26 ERROR 404: NOT FOUND.

